In [151]:
from db import call_ignition, call_db_json, insert_many_with_df, update_ignition
from queries import hk_flag, get_set_flag
from util_func import get_mondays, BlockKanbanStorage, BlockCoverage
from datetime import date, timedelta
import pandas as pd
import numpy as np
import bisect
block_kanban = BlockKanbanStorage()
block_coverage = BlockCoverage()

from main import ten_min_job
ten_min_job()
clamp = 10
kerf = 10
bkerf = 10


execute_many() done
12:44:20  10MIN update: processing, stock, demand, hk_unreleased, generating_missed_mo_batch


In [284]:
from db import call_ignition, call_db_json, insert_many_with_df, update_ignition
from queries import hk_flag, get_set_flag
from util_func import get_mondays, BlockKanbanStorage, BlockCoverage
from datetime import date, timedelta
import pandas as pd
import numpy as np
import bisect

block_kanban = BlockKanbanStorage()
block_coverage = BlockCoverage()

def block_demand(force_update=False):
    today = date.today()
    query_day = 168 - timedelta(today.weekday()).days
    sql = f"""
    SELECT product_family, product_model, item_description, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number, facility_id
        FROM manufacturing_orders
        INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
        WHERE order_status = 10
            AND order_release_code = 5
            AND facility_id in  ('A0010', 'A0045', 'E0010', 'J0005')
            AND product_model IS NOT NULL
            AND reference_number IS NOT NULL
            AND item_description like '%BLOCK%'
            AND order_scheduled_due <= CURRENT_DATE + {query_day}
        GROUP BY (product_family, product_model, product_variant, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due, facility_id)    
    """
    return call_db_json(sql)


def check_date_group(dt, mondays):
    index = bisect.bisect_right(mondays, dt.date())
    if index == len(mondays):
        return 25
    else:
        return index


# grouped_df
def get_name(row, type_=None):
    adder = "X" if row.product_family == "HSR" else ""
    product_variant = row["product_variant"]
    is_hass = ''    
    if type_ == 'STOCK':
        if 'GP/GH' in row["item_description"]:
            is_hass = ' HAAS'
    if type_ == 'DEMAND':
        if 'GP/GH' in row["item_description"]:
            is_hass = ' HAAS'

    if row["product_family"] == 'SHW':
        product_variant = row["product_variant"].replace("N", "")
    return (
        row["product_family"]
        + row["product_model"]
        + adder
        + product_variant
        + row["is_M"]
    )


def get_block_process():
    block_proc_sql = """
    SELECT
        order_number,
        product_family,
        product_model,
        product_variant,
        item_description,    
        order_quantity,
        printed_due,
        item_number,
        (
        SELECT
            string_agg(facility_id, ' ')
        FROM manufacturing_order_processes
        WHERE manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ) facility,
        (SELECT
            facility_id
        FROM manufacturing_order_processes
        WHERE operation_status != '40'
            AND manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ORDER BY operation_sequence
        LIMIT 1) facility_id,
        reference_number,
        order_scheduled_due,
        (SELECT
            time_in
        FROM manufacturing_order_logs
        WHERE manufacturing_order_logs.order_number = manufacturing_orders.order_number
        ORDER BY time_in DESC
        LIMIT 1) time_out
        FROM manufacturing_orders
        WHERE manufacturing_orders.order_status ='40'
            AND manufacturing_orders.order_release_code = 5
            AND product_model IS NOT NULL
            AND product_family IS NOT NULL
            AND product_variant IS NOT NULL
            AND item_description NOT LIKE '%+%'
            AND (reference_number LIKE 'HK%' or reference_number IN ('GY2', 'GY3', 'GY4'))
            AND product_block_count > 0
            AND product_family != 'TS'
        ORDER BY order_number
    """
    proc_db = call_db_json(block_proc_sql)
    pdf = pd.DataFrame(proc_db)
    pdf["is_M"] = pdf["item_description"].apply(lambda x: "M" if " M " in x else "")
    pdf["item_type"] = pdf.apply(get_name, axis=1)
    pdf["gy"] = pdf["reference_number"].apply(lambda x: "HK" if "HK" in x else "GY")
    rename = {"order_quantity": "qty", "order_number": "mos"}
    pdf = pdf.rename(columns=rename)
    p_groupby = pdf.groupby(["item_type", "facility_id", "gy"])
    p_grouped_df = p_groupby["qty"].sum()
    p_grouped_df = p_grouped_df.reset_index().set_index("item_type")
    p_grouped_df = p_grouped_df.pivot_table(
        index=["item_type", "facility_id"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    p_grouped_df.columns = ["item_type", "facility_id", "gy_qty", "hk_qty"]
    p_grouped_df["gy_qty"].fillna(0, inplace=True)
    p_grouped_df["hk_qty"].fillna(0, inplace=True)
    pdf = pdf[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "facility_id",
            "gy",
        ]
    ]
    return p_grouped_df, pdf


def check_stock_type(row):
    if "SET" in row.warehouse_location:
        return "SET"
    elif "HOLED" in row.item_description:
        return "HOLED"
    else:
        return ""


def gy4_concat(row):
    qty = row["GY4_gk"] + row["GY4_set"]
    combined_string = str(row["GY4_gk"]) + "/" + str(row["GY4_set"])
    return np.array([qty, combined_string])


def gy2_concat(row):
    qty = row["GY2"] + row["GY2_HOLED"]
    combined_string = str(row["GY2"]) + "/" + str(row["GY2_HOLED"])
    return np.array([qty, combined_string])

def drawn_rail_quantity(row, drawn_map):    
    drawn_row = drawn_map[(drawn_map['item_type'] == row.item_type) & (drawn_map['length'] == row.product_length)]
    # print(row.quantity)
    if drawn_row.empty:
        unit_length = drawn_map[(drawn_map['item_type'] == row.item_type)]['unit_length'].values[0]
        qty_per_drawn = int(row.product_length // (unit_length * 1.1))                
        return row.quantity * qty_per_drawn
    else:
        return drawn_row['cut_qty'].values[0] * row.quantity

def get_block_stock():
    block_stock_sql = """
        SELECT item_number, item_description, product_family, product_model, product_variant, product_block_count, product_length, warehouse_location, quantity, virtual_location
            FROM current_wip_inventory
            WHERE block=true
                AND product_variant IS NOT NULL
                AND warehouse_location != 'INTRAN'
        """
    s_df = pd.DataFrame(call_db_json(block_stock_sql))    
    # 
    s_df["is_M"] = s_df["item_description"].apply(lambda x: "M" if (" M " in x) or (x.split(" ")[0][-1] == 'M') else "")
    s_df["item_type"] = s_df.apply(get_name, axis=1, args=['stock'])
    drawn_map_sql = """
        SELECT * from initial_release.drawn_block_map
        """
    drawn_map = pd.DataFrame(call_ignition(drawn_map_sql))

    s_df["warehouse_location"] = s_df.apply(check_stock_type, axis=1)    
    s_df[['product_block_count', 'product_length', 'quantity']] = s_df[['product_block_count', 'product_length', 'quantity']].astype(int)
    s_df.loc[(s_df['virtual_location'] == 'etc') & (~s_df['warehouse_location'].isin(['GYCOAT', 'INTRAN'])), 'virtual_location'] = 'GY4'
    s_df = s_df.loc[df['virtual_location'] != 'etc']


    s_df.loc[s_df['item_description'].str.contains('DRAWN'), 'quantity'] = s_df[s_df['item_description'].str.contains('DRAWN')].apply(drawn_rail_quantity, axis=1, args=[drawn_map])
    s_groupby = s_df.groupby(
        ["item_type", "virtual_location", "warehouse_location"]
    )    
    qty = s_groupby["quantity"].sum().reset_index()
    s_df = qty.pivot_table(
        index=["item_type"],
        columns=["warehouse_location", "virtual_location"],
        values="quantity",
    )
    s_df = s_df.fillna(0).reset_index()
    s_df.columns = ["item_type", "GY1", "GY2", "GY3", "GY4_gk", "GY2_HOLED", "GY4_set"]
    s_df[["GY4_gk", "GY4_set", "GY2", "GY2_HOLED"]] = s_df[
        ["GY4_gk", "GY4_set", "GY2", "GY2_HOLED"]
    ].astype(int)
    s_df["GY4"] = s_df.apply(gy4_concat, axis=1)
    s_df["GY2"] = s_df.apply(gy2_concat, axis=1)
    s_df = s_df.drop(columns=["GY4_gk", "GY4_set", "GY2_HOLED"])
    s_df = pd.melt(
        s_df, id_vars="item_type", var_name="facility_id", value_name="order_quantity"
    )
    return s_df


def gy_checker(facility_id):
    if facility_id == "J0005":
        return "GY4"
    elif facility_id == "E0010":
        return "GY3"
    else:
        return "GY2"


def get_block_demand():
    mondays = get_mondays(23)
    mondays = [i.date() for i in mondays]

    demand = pd.DataFrame(block_demand())
    try:
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    except:
        get_set_flag(True)
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    demand["order_scheduled_due"] = pd.to_datetime(demand["order_scheduled_due"])

    demand["week"] = demand["order_scheduled_due"].apply(
        check_date_group, args=[mondays]
    )
    demand["is_M"] = demand["item_description"].apply(
        # lambda x: "M" if (" M " in x) or (x.split(" ")[0][-1] == 'M') else ""
        lambda x: "M" if " M " in x else ""
    )
    demand = demand.rename(columns={"facility_id": "gy"})

    demand["gy"] = demand["gy"].apply(gy_checker)
    demand["item_type"] = demand.apply(get_name, axis=1)
    d_groupby = demand.groupby(
        ["item_type", "week", "gy"]
    )

    qty = d_groupby["qty"].sum()
    demand_grouped = qty.reset_index()
    # demand_grouped["item_type"] = demand_grouped.apply(get_name, axis=1)
    demand_grouped = demand_grouped.drop(
        columns=["product_family", "product_model", "product_variant", "is_M"]
    )
    demand_grouped = demand_grouped.pivot_table(
        index=["item_type", "week"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    demand_grouped.columns = ["item_type", "week", "gy2_qty", "gy3_qty", "gy4_qty"]
    # demand_grouped[["gy2_qty", "gy3_qty","gy4_qty"]].fillna(0, inplace=True)
    demand_grouped["gy2_qty"].fillna(0, inplace=True)
    demand_grouped["gy3_qty"].fillna(0, inplace=True)
    demand_grouped["gy4_qty"].fillna(0, inplace=True)
    # demand_grouped.fillna(0, inplace=True)
    demand = demand[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "week",
            "hk_flag",
            "gy",
        ]
    ].copy()
    demand["block_covered"] = False
    demand = demand.sort_values("order_scheduled_due")
    return demand_grouped, demand, mondays


def block_processing_write_databse(processing, proc_stoc_due_map):
    processing = processing.merge(
        proc_stoc_due_map[["item_type", "facility_id", "due"]],
        on=["item_type", "facility_id"],
        how="left",
    )
    processing.loc[
        processing["order_scheduled_due"].isnull(), "order_scheduled_due"
    ] = processing["due"]
    processing = processing.fillna("")
    processing["order_scheduled_due"] = processing["order_scheduled_due"].apply(
        lambda x: x.strftime("%Y-%m-%d") if x != "" else None
    )
    processing = (
        processing.drop(columns=["gy", "due"])
        .rename(columns={"mos": "mo"})
        .fillna("NO DUE")
    )
    processing = processing.groupby(["facility_id"]).apply(
        lambda x: x.sort_values("order_scheduled_due")
    )
    processing.reset_index(drop=True, inplace=True)
    processing["facility_priority"] = processing.groupby(["facility_id"]).cumcount() + 1
    table = "ignition.initial_release.block_priority"
    delete_sql = f"TRUNCATE {table}; DELETE FROM {table}"    
    update_ignition(delete_sql)
    insert_many_with_df("ignition", processing, "initial_release.block_priority")


def generate_block_kanban():
    facility_list = [
        "GY1",
        "A0020",
        "A0030",
        "GY2",
        "A0050",        
        "A0070",
        "GY3",
        "E0020",
        "F0020",
        "E0030",
        "E0050",
        "GY4",
    ]
    facility_dict = {}
    for i, v in enumerate(facility_list):
        facility_dict[v] = i
    demand_len = 25
    proc_stoc_len = len(facility_list)
    demand_grouped, demand, mondays = get_block_demand()
    processing_grouped, processing = get_block_process()
    stock_df = get_block_stock()
    proc_stoc = pd.concat([processing_grouped, stock_df]).reset_index().fillna(0)
    proc_stoc = proc_stoc[proc_stoc["facility_id"].isin(facility_list)]
    item_types = pd.unique(demand_grouped.item_type)
    res = []
    proc_stoc_due_map = pd.DataFrame(
        columns=["item_type", "facility_id", "order_quantity"]
    )
    for item in item_types:
        local_demand_grouped = demand_grouped[demand_grouped["item_type"] == item]
        local_proc_stoc = proc_stoc[proc_stoc["item_type"] == item]
        temp_list = [0] * (demand_len + proc_stoc_len)
        gy_stock = [0, 0, 0]
        gy_exh = [False, False, False]
        exh_index = [99, 99, 99]
        # gy_stock = [0, 0]
        # gy_exh = [False, False]
        # exh_index = [99, 99]
        GY2, GY4 = 0, 0
        for row in local_proc_stoc.itertuples():
            index = facility_list.index(row.facility_id) + demand_len
            local_processing = processing[
                (processing["item_type"] == item)
                & (processing["facility_id"] == row.facility_id)
            ]
            ### GY4_stock for GK/SET seperation
            additional_stock = None
            ### GY stocks

            if "GY" in row.facility_id:
                if row.facility_id in "GY2":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 2 : index]
                        ]
                    )
                    gy_stock[0] = int(row.order_quantity[0]) + proc_sum
                    GY2 = row.order_quantity[0]
                    additional_stock = row.order_quantity[1]
                elif row.facility_id == "GY3":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 2 : index]
                        ]
                    )
                    gy_stock[1] = row.order_quantity + proc_sum
                elif row.facility_id == "GY4":
                    proc_sum = sum(
                        [
                            obj.get("gy").get("qty") if obj != 0 else 0
                            for obj in temp_list[index - 4 : index]
                        ]
                    )
                    gy_stock[2] = int(row.order_quantity[0]) + proc_sum
                    GY4 = row.order_quantity[0]
                    additional_stock = row.order_quantity[1]
                temp_list[index] = {
                    "additional": additional_stock,
                    "qty": int(row.order_quantity[0])
                    if row.facility_id not in ["GY1", "GY3"] 
                    else row.order_quantity,
                    "proc_sum": proc_sum if row.facility_id != 'GY1' else 0,
                    "type": "STOCK",
                }
            ### block processing
            else:
                temp_list[index] = {
                    "gy": {
                        "df": local_processing[local_processing["gy"] == "GY"].to_dict(
                            orient="records"
                        ),
                        "qty": row.gy_qty,
                    },
                    "hk": {
                        "df": local_processing[local_processing["gy"] == "HK"].to_dict(
                            orient="records"
                        ),
                        "qty": row.hk_qty,
                    },
                    "type": "PROCESSING",
                }

        local_proc_stoc = local_proc_stoc.sort_values(
            "facility_id", key=lambda x: x.map(facility_dict), ascending=False
        )
        local_proc_stoc.loc[
            local_proc_stoc["facility_id"] == "GY4", "order_quantity"
        ] = GY4
        local_proc_stoc.loc[
            local_proc_stoc["facility_id"] == "GY2", "order_quantity"
        ] = GY2
        local_proc_stoc["order_quantity"] = local_proc_stoc["order_quantity"].astype(
            float
        )
        local_proc_stoc["order_quantity"] = (
            local_proc_stoc["order_quantity"] + local_proc_stoc["gy_qty"]
        )
        local_proc_stoc = local_proc_stoc[
            ["item_type", "facility_id", "order_quantity"]
        ]
        local_proc_stoc["due"] = None
        for row in local_demand_grouped.itertuples():
            qty = [row.gy2_qty, row.gy3_qty, row.gy4_qty]
            breakpoint_values = [False, False, False]
            local_demand = demand[
                (demand["item_type"] == item) & (demand["week"] == row.week)
            ]
            # print(local_demand)
            ### calculate for each GY
            week_date = mondays[row.week - 1]
            if row.gy4_qty > 0:
                demand_qty = row.gy4_qty
                gy4_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[1:])
                ]                
                for proc in gy4_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if (
                            "GY" not in proc.facility_id
                            and local_proc_stoc.loc[proc[0], "due"] == None
                        ):
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            if row.gy3_qty > 0:
                demand_qty = row.gy3_qty
                gy3_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[1:7])
                ]
                for proc in gy3_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            if row.gy2_qty > 0:
                demand_qty = row.gy2_qty
                gy2_proc_stock = local_proc_stoc[
                    local_proc_stoc["facility_id"].isin(facility_list[1:4])
                ]
                for proc in gy2_proc_stock.itertuples():
                    surplus = proc.order_quantity - demand_qty
                    if surplus < 0:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = 0
                        demand_qty -= proc.order_quantity
                    else:
                        if local_proc_stoc.loc[proc[0], "due"] == None:
                            local_proc_stoc.loc[proc[0], "due"] = week_date
                        local_proc_stoc.loc[proc[0], "order_quantity"] = surplus
                        break

            for i, gy in enumerate(["GY2", "GY3", "GY4"]):
                gy_stock[i] -= qty[i]
                ### stock is negative
                if gy_stock[i] < 0:
                    ### stock gets negative
                    if gy_exh[i] == False:
                        ### exhaustion flag turn on
                        gy_exh[i] = True
                        ### exhaustion week record
                        exh_index[i] = row.week
                        covered_qty = qty[i] + gy_stock[i]
                        if covered_qty != 0:
                            breakpoint_values[i] = [covered_qty, -gy_stock[i]]
                            ### check which demand mos are covered. prioritize set order to block GK orders.
                            for local_demand_row in local_demand[
                                local_demand["hk_flag"] == "RBS"
                            ].itertuples():
                                covered_qty -= local_demand_row.qty
                                if covered_qty >= 0:
                                    demand.loc[
                                        local_demand_row[0], "block_covered"
                                    ] = True
                                else:
                                    break
                ### stock remains positive then block_covered true
                else:
                    demand.loc[
                        (demand["item_type"] == item)
                        & (demand["week"] == row.week)
                        & (demand["gy"] == gy),
                        "block_covered",
                    ] = True

            temp_list[row.week - 1] = {
                "gy2": {
                    "df": local_demand[local_demand["gy"] == "GY2"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy2_qty,
                    "exh": gy_exh[0],
                    "breakingpoint": breakpoint_values[0],
                },
                "gy3": {
                    "df": local_demand[local_demand["gy"] == "GY3"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy3_qty,
                    "exh": gy_exh[1],
                    "breakingpoint": breakpoint_values[1],
                },
                "gy4": {
                    "df": local_demand[local_demand["gy"] == "GY4"].to_dict(
                        orient="records"
                    ),
                    "qty": row.gy4_qty,
                    "exh": gy_exh[2],
                    "breakingpoint": breakpoint_values[2],
                },
                "type": "DEMAND",
            }
        proc_stoc_due_map = pd.concat([proc_stoc_due_map, local_proc_stoc])
        item_dict = {"item_name": item, "data": temp_list, "exh_index": exh_index}
        res.append(item_dict)
    block_coverage.set_data(demand)
    block_processing_write_databse(processing, proc_stoc_due_map)
    return {"res": res, "mondays": [i.strftime("%m-%d") for i in mondays]}


def get_block_kanban(update=False):
    if block_kanban.empty or update:
        block_kanban.set_data(generate_block_kanban())
    return block_kanban.get_data()


In [285]:

hass_list = ['HSR35C1SSC0E(GP) BLOCK',
'HSR45C1SSC0S(GP) BLOCK',
'SHS30V1SSC0S',
'SHS30LV1SSC1S']
if any([i in 'HSR35C1SSC0E(GP) BLOCK' for i in hass_list]):
    print('a')


a


In [286]:
hass_list = ['HSR35C1SSC0E',
'HSR45C1SSC0S',
'SHS30V1SS',
'SHS30LV1SSC1S']
hass_map = {'HSR35C1SSC0E':'HSR35C HAAS',
'HSR45C1SSC0S':'HSR45CA HAAS',
'SHS30V1SS':'SHS30V HAAS',
'SHS30LV1SSC1S':'SHS30LV HAAS'}

def get_name(row, type_=None):
    adder = "X" if row.product_family == "HSR" else ""
    product_variant = row["product_variant"]
    is_hass = ''        

    if type_ == 'STOCK':
        if 'GP/GH' in row["item_description"]:
            is_hass = ' HAAS'
    if type_ == 'DEMAND':
        if 'SHS30LV' in row['item_description']:
            # print(row['item_description'], row['mos'])
            print(row)
        if any([i in row["item_description"] for i in hass_list]):
            is_hass = ' HAAS'            

    if row["product_family"] == 'SHW':
        product_variant = row["product_variant"].replace("N", "")
    return (
        row["product_family"]
        + row["product_model"]
        + adder
        + product_variant
        + row["is_M"]
        + is_hass
    )

def get_block_demand():
    mondays = get_mondays(23)
    mondays = [i.date() for i in mondays]

    demand = pd.DataFrame(block_demand())
    try:
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    except:
        get_set_flag(True)
        demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
    demand["order_scheduled_due"] = pd.to_datetime(demand["order_scheduled_due"])

    demand["week"] = demand["order_scheduled_due"].apply(
        check_date_group, args=[mondays]
    )
    demand["is_M"] = demand["item_description"].apply(
        # lambda x: "M" if (" M " in x) or (x.split(" ")[0][-1] == 'M') else ""
        lambda x: "M" if " M " in x else ""
    )
    
    demand = demand.rename(columns={"facility_id": "gy"})

    demand["gy"] = demand["gy"].apply(gy_checker)
    print(demand.columns)
    demand["item_type"] = demand.apply(get_name, axis=1, args=["DEMAND"])
    d_groupby = demand.groupby(
        ["item_type", "week", "gy"]
    )

    qty = d_groupby["qty"].sum()
    demand_grouped = qty.reset_index()
    # demand_grouped["item_type"] = demand_grouped.apply(get_name, axis=1)
    # demand_grouped = demand_grouped.drop(
    #     columns=["product_family", "product_model", "product_variant", "is_M"]
    # )
    demand_grouped = demand_grouped.pivot_table(
        index=["item_type", "week"], columns="gy", values="qty", aggfunc="sum"
    ).reset_index()
    demand_grouped.columns = ["item_type", "week", "gy2_qty", "gy3_qty", "gy4_qty"]
    # demand_grouped[["gy2_qty", "gy3_qty","gy4_qty"]].fillna(0, inplace=True)
    demand_grouped["gy2_qty"].fillna(0, inplace=True)
    demand_grouped["gy3_qty"].fillna(0, inplace=True)
    demand_grouped["gy4_qty"].fillna(0, inplace=True)
    # demand_grouped.fillna(0, inplace=True)
    demand = demand[
        [
            "item_description",
            "order_scheduled_due",
            "qty",
            "reference_number",
            "mos",
            "item_type",
            "week",
            "hk_flag",
            "gy",
        ]
    ].copy()
    demand["block_covered"] = False
    demand = demand.sort_values("order_scheduled_due")
    return demand_grouped, demand, mondays
    


In [ ]:
a, b, c = get_block_demand()

In [237]:
backup_sql = """
    SELECT product_family, product_model, order_customer_name, product_block_count, item_description, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number        FROM manufacturing_orders
        WHERE order_status = 10
            AND order_release_code = 5
            AND reference_number like 'HK%'
            AND product_family = 'HSR'
            AND product_model = '35'
            AND product_variant = 'C'
            AND order_scheduled_due <= CURRENT_DATE + 164
        GROUP BY (product_family, product_model, order_customer_name, product_block_count, product_variant, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due)
"""


In [256]:
sql = """
    SELECT product_family, product_model, order_customer_name, product_block_count, item_description, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number        FROM manufacturing_orders
        WHERE order_status = 10
            AND order_release_code = 5
            AND order_customer_name = 'HAAS AUTOMATION'
            AND reference_number like 'HK%'
            AND product_block_count > 0    
            AND order_scheduled_due <= CURRENT_DATE + 164
        GROUP BY (product_family, product_model, order_customer_name, product_block_count, product_variant, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due)
    """
df = pd.DataFrame(call_db_json(sql))
df['qty'] = df['qty'] * df['product_block_count']
df['type'] = df['product_family'] + df['product_model']
df = df.drop(columns=['order_customer_name', 'item_number'])
pd.unique(df['type'])

array(['HDR35', 'HSR30', 'HSR35', 'HSR45', 'SHS30'], dtype=object)

In [267]:
df[df['type'] == 'SHS30']

,product_family,product_model,product_block_count,item_description,product_variant,order_scheduled_due,printed_due,qty,mos,reference_number,type
39,SHS,30,2,SHS30LV2SSC0S+1040LP,LV,2023-09-06,2023-09-06,80.0,"[M217024, M217027, M217026, M217025]",HK321359,SHS30
40,SHS,30,2,SHS30LV2SSC0S+1040LP,LV,2023-09-06,2023-09-06,80.0,"[M217037, M217038, M217039, M217036]",HK321360,SHS30
41,SHS,30,2,SHS30LV2SSC0S+830LP/50-0197,LV,2023-10-16,2023-10-10,16.0,"[M239453, M239452]",HK334650,SHS30
42,SHS,30,2,SHS30LV2SSC0S+830LP/50-0197,LV,2023-10-16,2023-10-11,36.0,"[M239458, M239457, M239456]",HK334650,SHS30
43,SHS,30,2,SHS30LV2SSC0S+830LP/50-0197,LV,2023-10-16,2023-10-12,36.0,"[M239462, M239464, M239463]",HK334650,SHS30
44,SHS,30,2,SHS30LV2SSC0S+830LP/50-0197,LV,2023-10-16,2023-10-13,36.0,"[M239468, M239469, M239470]",HK334650,SHS30
45,SHS,30,2,SHS30LV2SSC0S+830LP/50-0197,LV,2023-10-16,2023-10-16,36.0,"[M239474, M239476, M239475]",HK334650,SHS30
46,SHS,30,2,SHS30LV2SSC0S+930LP,LV,2022-12-16,2022-12-16,32.0,"[M149986, M149987]",HK281975,SHS30
47,SHS,30,2,SHS30LV2SSC0S+930LP,LV,2023-09-28,2023-09-26,180.0,[M204162],HK312707,SHS30
48,SHS,30,2,SHS30LV2SSC0S+930LP,LV,2023-09-28,2023-09-27,180.0,[M204163],HK312707,SHS30


In [297]:

def block_demand(force_update=False):
    today = date.today()
    query_day = 168 - timedelta(today.weekday()).days
    sql = f"""SELECT item_description, product_family, order_customer_name, product_model, product_block_count, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number, facility_id
            FROM manufacturing_orders
            INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
            WHERE order_status = 10
                AND order_release_code = 5                        
                AND facility_id in  ('A0010', 'A0045', 'E0010', 'J0005', 'K0015', 'K0020')            
                AND product_model IS NOT NULL
                AND reference_number IS NOT NULL
                AND product_block_count > 0
                AND order_scheduled_due <= CURRENT_DATE + {query_day}
            GROUP BY (product_family, item_description, product_model, order_customer_name, product_variant, product_block_count, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due, facility_id)"""
    df = pd.DataFrame(call_db_json(sql))
    df['mos'] = df['mos'].apply(lambda x: sorted(x))
    df = df.sort_values('facility_id')
    df = df.drop_duplicates(subset=['mos'], keep='first')
    df['qty'] = df['qty'] * df['product_block_count']
    df['type'] = df['product_family'] + df['product_model']     
    return df

In [ ]:

def gy_checker(facility_id):
    if facility_id == "J0005":
        return "GY4"
    elif facility_id == "E0010":
        return "GY3"
    elif facility_id in ['K0015', 'K0020']:
        return "GYHAAS"
    else:
        return "GY2"

hass_list = ['HSR35C1SSC0E',
'HSR45C1SSC0S',
'SHS30V1SS',
'SHS30LV1SSC1S']
hass_map = {'HSR35C1SSC0E':'HSR35C HAAS',
'HSR45C1SSC0S':'HSR45CA HAAS',
'SHS30V1SS':'SHS30V HAAS',
'SHS30LV1SSC1S':'SHS30LV HAAS'}


def get_name(row, type_=None):
    adder = "X" if row.product_family == "HSR" else ""
    product_variant = row["product_variant"]
    is_hass = ''        

    if type_ == 'STOCK':
        if 'GP/GH' in row["item_description"]:
            is_hass = ' HAAS'
    if type_ == 'DEMAND':        
        if row['order_customer_name'] == 'HAAS AUTOMATION':
            is_hass = ' HAAS'            

    if row["product_family"] == 'SHW':
        product_variant = row["product_variant"].replace("N", "")
    print(row)
    return (
        row["product_family"]
        + row["product_model"]
        + adder
        + product_variant
        + row["is_M"]
        + is_hass
    )


# def get_block_demand():
mondays = get_mondays(23)
mondays = [i.date() for i in mondays]

demand = block_demand()
demand.loc[demand['order_customer_name'] == 'HAAS AUTOMATION', 'hk_flag'] = 'HAAS'
demand.loc[demand['hk_flag'] != 'HAAS', 'hk_flag'] = demand.loc[demand['hk_flag'] != 'HAAS', 'reference_number'].apply(lambda x: hk_flag.flag[x])

try:
    demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
except:
    get_set_flag(True)
    demand["hk_flag"] = demand["reference_number"].apply(lambda x: hk_flag.flag[x])
demand["order_scheduled_due"] = pd.to_datetime(demand["order_scheduled_due"])

demand["week"] = demand["order_scheduled_due"].apply(
    check_date_group, args=[mondays]
)
demand["is_M"] = demand["item_description"].apply(
    # lambda x: "M" if (" M " in x) or (x.split(" ")[0][-1] == 'M') else ""
    lambda x: "M" if " M " in x else ""
)
demand = demand.rename(columns={"facility_id": "gy"})

demand["gy"] = demand["gy"].apply(gy_checker)
demand["item_type"] = demand.apply(get_name, axis=1)
d_groupby = demand.groupby(
    ["item_type", "week", "gy"]
)

qty = d_groupby["qty"].sum()
demand_grouped = qty.reset_index()
# demand_grouped["item_type"] = demand_grouped.apply(get_name, axis=1)
demand_grouped = demand_grouped.drop(
    columns=["product_family", "product_model", "product_variant", "is_M"]
)
demand_grouped = demand_grouped.pivot_table(
    index=["item_type", "week"], columns="gy", values="qty", aggfunc="sum"
).reset_index()
demand_grouped.columns = ["item_type", "week", "gy2_qty", "gy3_qty", "gy4_qty"]
# demand_grouped[["gy2_qty", "gy3_qty","gy4_qty"]].fillna(0, inplace=True)
demand_grouped["gy2_qty"].fillna(0, inplace=True)
demand_grouped["gy3_qty"].fillna(0, inplace=True)
demand_grouped["gy4_qty"].fillna(0, inplace=True)
# demand_grouped.fillna(0, inplace=True)
demand = demand[
    [
        "item_description",
        "order_scheduled_due",
        "qty",
        "reference_number",
        "mos",
        "item_type",
        "week",
        "hk_flag",
        "gy",
    ]
].copy()
demand["block_covered"] = False
demand = demand.sort_values("order_scheduled_due")
    # return demand_grouped, demand, mondays
# a, b, c = get_block_demand()

In [363]:
query_day = 164
set_sql =f"""
    SELECT product_family, product_model, item_description, product_variant, order_scheduled_due, printed_due, order_quantity, order_number, item_number, reference_number
        FROM manufacturing_orders        
        WHERE order_status = 10
            AND item_description like '%+%'
            AND order_scheduled_due <= CURRENT_DATE + {query_day}
            AND order_release_code = 5
            AND product_family NOT IN ('S-UNIT', 'GL', 'KR', 'TY')
            AND reference_number IS NOT NULL
    """
set_df = pd.DataFrame(call_db_json(set_sql))

In [364]:
set_reference_numbers = set_df['reference_number'].to_list()
set_reference_string = str(set_reference_numbers)[1:-1]
set_block_sql = f"""SELECT product_family, product_model, order_status, item_description, product_variant, order_scheduled_due, printed_due, order_quantity, manufacturing_orders.order_number, item_number, reference_number, facility_id
        FROM manufacturing_orders
        INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
        WHERE reference_number in ({set_reference_string})            
            AND product_model IS NOT NULL
            AND reference_number IS NOT NULL
            AND item_description like '%BLOCK%'            
            AND order_status = 5
    """
# sql = f"""
#     SELECT product_family, product_model, item_description, product_variant, order_scheduled_due, printed_due, SUM(order_quantity) AS qty, ARRAY_AGG(manufacturing_orders.order_number) as MOS, item_number, reference_number, facility_id
#         FROM manufacturing_orders
#         INNER join manufacturing_order_processes on manufacturing_orders.order_number = manufacturing_order_processes.order_number
#         WHERE item_description like '%BLOCK%'
#             AND order_release_code = 5            
#             AND product_model IS NOT NULL
#             AND reference_number IS NOT NULL
#             AND item_description like '%BLOCK%'
#             AND order_scheduled_due <= CURRENT_DATE + {query_day}
#         GROUP BY (product_family, product_model, product_variant, item_description, item_number, printed_due, reference_number, order_scheduled_due, printed_due, facility_id)    
# """
set_block_df = pd.DataFrame(call_db_json(set_block_sql))
only_set = set_df[~set_df['reference_number'].isin(block_df['reference_number'].to_list())].copy()
block_df = block_df[block_df['order_status'] == 10].drop(columns=['order_status'])
only_set['facility_id'] = 'J0005'

In [367]:
block_df.groupby('reference_number').



,product_family,product_model,item_description,product_variant,order_scheduled_due,printed_due,order_quantity,order_number,item_number,reference_number,facility_id
19,SHS,35,SHS35LR C1S BLOCK,LR,2023-09-15,2023-09-15,4.0,M196656,502302294263400,HK307752,A0045
26,SHS,35,SHS35LR C1S BLOCK,LR,2023-09-08,2023-09-08,4.0,M173127,502302294263400,HK293148,A0045
27,SHS,35,SHS35LR C1S BLOCK,LR,2023-09-08,2023-09-08,8.0,M173128,502302294263400,HK293148,A0045
28,SHS,35,SHS35LR C1S BLOCK,LR,2023-09-08,2023-09-08,8.0,M173129,502302294263400,HK293148,A0045
29,SHS,35,SHS35LR C1S BLOCK,LR,2023-09-22,2023-09-22,4.0,M173141,502302294263400,HK293170,A0045
...,...,...,...,...,...,...,...,...,...,...,...
1918,SHS,25,SHS25LV C1 BLOCK,LV,2023-10-02,2023-10-02,4.0,M243569,402325400202000,HK337257,J0005
1919,SHS,25,SHS25C C1 BLOCK,C,2023-10-06,2023-10-03,8.0,M243571,402325100202000,HK337287,J0005
1920,SSR,15X,SSR15XW BLOCK,W,2024-01-12,2024-01-12,8.0,M243578,400516100002000,HK337198,J0005
1921,SHS,45,SHS45R C1 BLOCK,R,2023-08-31,2023-08-28,4.0,M243581,402345500202000,HK337256,J0005


In [361]:
block_df.head()

,product_family,product_model,order_status,item_description,product_variant,order_scheduled_due,printed_due,order_quantity,order_number,item_number,reference_number,facility_id
0,SHS,35,40,SHS35LR C1S BLOCK,LR,2023-08-16,2023-08-16,8.0,M196632,502302294263400,HK307751,A0045
1,SHS,35,40,SHS35LR C1S BLOCK,LR,2023-08-16,2023-08-16,8.0,M196631,502302294263400,HK307751,A0045
2,SHS,35,40,SHS35LR C1S BLOCK,LR,2023-08-16,2023-08-16,8.0,M196634,502302294263400,HK307751,A0045
3,SHS,35,40,SHS35LR C1S BLOCK,LR,2023-08-16,2023-08-16,8.0,M196635,502302294263400,HK307751,A0045
4,SHS,35,40,SHS35LR C1S BLOCK,LR,2023-08-25,2023-08-25,4.0,M173108,502302294263400,HK293169,A0045
